In [ ]:
%%html
<style>
    #notebook-container{
    width: 100%
    }
</style>

In [ ]:
import altair as alt
import pandas as pd

from altair import datum
from vega_datasets import data
from geopy.geocoders import Nominatim

# alt.renderers.enable('notebook')

In [ ]:
alt.data_transformers.enable('default', max_rows=None) 

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/kai-wren/UCU_DataViz/master/gapminder.csv')
df = pd.read_csv('scrubbed_UFO.csv')
df = df.rename(columns={'longitude ': 'longitude'})
df.replace(to_replace='&#39', value='\'', inplace=True, regex=True)
df.replace(to_replace='&#44', value=',', inplace=True, regex=True)
df['type'] = "UFO"

df.head()

In [ ]:
df_meteorite = pd.read_csv('meteorite-landings.csv')
df_meteorite['type'] = "Meteorite"

df_meteorite.head()

Question to be answered: <br>
- I would like to match data with meteorite landing dataset to check that maybe some spotted UFOs were just meteorites and whether or not the concentration of spotted UFO coincides with the concentration of observed meteorites on the global map.
- Are there places on Earth which have more UFOs spotted than in other places. <br>



In [ ]:
year_slider = alt.binding_range(min=1906, max=2014, step=1, name='Year')
year_selector = alt.selection_single(name="YearSelector", fields=['selected_year'],
                                bind=year_slider, init={'selected_year': 1995})

base_meteorite = alt.Chart(df_meteorite
).transform_filter(datum.year <= year_selector.selected_year).transform_filter(datum.year >= year_selector.selected_year)

base_ufo = alt.Chart(df
).transform_calculate(year = "year(datum.datetime)"
).transform_filter(datum.year <= year_selector.selected_year
).transform_filter(datum.year >= year_selector.selected_year
)

source = alt.topo_feature(data.world_110m.url, 'countries')

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points_meteorite = base_meteorite.mark_circle(
    size=15,
).encode(
    latitude='reclat:Q',
    longitude='reclong:Q',
    color=alt.value('blue'),
#     tooltip=['type:N', 'year:N', 'name:N', 'mass:N'],
    opacity=alt.value(0.5)
)

points_ufo = base_ufo.mark_circle(
    size=15,
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    color=alt.value('red'),
#     tooltip=['type:N', 'shape:N', 'city:N', 'duration (hours/min):N', 'datetime:N'],
    opacity=alt.value(0.5)
).add_selection(year_selector)

year_back_ufo = base_ufo.mark_text(fontSize=130, fill='lightgray', align='right', baseline='top', dx=-300, dy=100).encode(
text=alt.Text('mean(year):N'),
)

year_back_meteorite = base_meteorite.mark_text(fontSize=130, fill='lightgray', align='right', baseline='top', dx=-300, dy=100).encode(
text=alt.Text('mean(year):N'),
)

(earth+year_back_ufo+year_back_meteorite+points_ufo+points_meteorite).properties(width=1200, height=600)

Question to be answered: <br>
Does the amount of spotted UFO increase with the course of time? This is especially interesting with regards to last 20-30 years with widespread communication technologies. <br>

Here I plot simple barchart to highlight an answer to the question. In addition I've added another chart which allows to see in which month and day how many observation happens. <br>

For interactivity, you could select one particular year or years and right hand side chart will be updated for selected years.

In [ ]:
base_ufo = alt.Chart(df)

selector = alt.selection_multi(encodings=['x'],empty='all')

bars = base_ufo.mark_bar().encode(
y=alt.Y('count()'),
x=alt.X('year(datetime):N'),
color = alt.condition(selector, alt.value('blue'), alt.value('gray'))
).add_selection(selector
).properties(width=1000, height=500)#.transform_calculate(year = "year(datum.datetime)").transform_filter(datum.year == 1902)

ticks = base_ufo.mark_point().encode(
y=alt.Y('month(datetime):N'),
x=alt.X('date(datetime):O'),
size=alt.Size('count()')
).transform_filter(selector
).properties(width=600, height=500)

bars | ticks

Question to be answered: <br>
Do types of spotted UFOs equally spread or only appears in certain locations.<br>

I've made simillar looking map and add some barcharts on the edges of it to indicate distribution across longitude and latitude. To allow some filtering I have added kind of interactive legend made from heatmap with indication of total count per each UFO type.

In [ ]:
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')
shape_selector=alt.selection_single(fields=['shape'])
leg_color=alt.Color('count()', legend=None)

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points = base_ufo.mark_circle(
    size=10
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    tooltip=['shape:N', 'city:N', 'latitude:Q', 'longitude:Q'],
).properties(width=1200, height=600).transform_filter(shape_selector)

lat_ufo = base_ufo.mark_bar().encode(
    y='latitude:Q',
    x='count()',
).properties(width=300, height=600).transform_filter(shape_selector)

lon_ufo = base_ufo.mark_bar().encode(
    y='count()',
    x='longitude:Q',
).properties(width=1200, height=300).transform_filter(shape_selector)

leg = base_ufo.mark_rect().encode(
y='shape:N',
color=alt.condition(shape_selector, leg_color, alt.value('gray'))
).properties(width=100, height=300).add_selection(shape_selector)

leg_text = base_ufo.mark_text().encode(
y='shape:N',
text='count()'
# color=alt.condition(shape_selector, leg_color, alt.value('gray'))
).properties(width=100, height=300)

(lon_ufo | (leg+leg_text) )& ((earth+points) | lat_ufo)

Question to be answered: <br>
How long UFO were observed per country in the dataset?

I was going to represent each country as bubble on world map size of which indicats how long overall duration per country. Unfortunately, I've discovered that country name is almost missing in the dataset. Hence I will need to enrich it first using some other services. For now Canada and UK indicate example how it should look like.

In [ ]:
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points = base_ufo.mark_circle(
    size=10
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    size=alt.Size('duration:Q'),
    tooltip=['country:N']
).transform_aggregate(
latitude='mean(latitude)',
longitude='mean(longitude)',
duration = 'sum(duration (seconds))',
groupby=['country']
)

(earth+points).properties(width=1200, height=600)

# test = base_ufo.mark_bar(
#     size=10
# ).encode(
#     x='country:N',
#     y='sum(duration (seconds)):Q',
# )
# # .transform_aggregate(
# # latitude='mean(latitude)',
# # longitude='mean(longitude)',
# # duration = 'sum(duration (seconds))',
# # groupby=['country']
# # )

# test

# Drafts

Question to be answered: <br>
Are there places on Earth which have more UFOs spotted than in other places. <br>

Here I am plotting a point indicating one record each. This allow to see how encounters are spread on world map. Points are located based on latitude and longitude values. Hence if those values are empty - points located wrongly in the top left corner. In the end density of points indicates places with bigger amounts of contacts. <br>

For interactivity I've added barchart with interval selection to limited amount of dots to be displayed.

In [ ]:
base_ufo = alt.Chart(df)

source = alt.topo_feature(data.world_110m.url, 'countries')

# slider = alt.binding_range(min=1900, max=2015, step=1, name='Year')
# selector = alt.selection_single(name="YearSelector", fields=['cutoff'],
#                                 bind=slider, init={'cutoff': 2015})

brush = alt.selection_interval(encodings=['x'],empty='all')

base_world = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
)

earth = base_world.project('naturalEarth1')

points = base_ufo.mark_circle(
    size=10,
    color='red'
).encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    tooltip=['shape:N', 'city:N', 'duration (hours/min):N', 'date posted:N']
).transform_filter(brush)
# .add_selection(selector).transform_calculate(year = "year(datum.datetime)").transform_filter(datum.year <= selector.cutoff)

area_ufo = base_ufo.mark_bar().encode(
x=alt.X('year(datetime):N'),
y=alt.Y('count()'),
opacity=alt.value(0.3)
)

area_ufo_notselected = area_ufo.add_selection(brush)
area_ufo_selected = area_ufo.mark_bar().encode(opacity=alt.value(1)).transform_filter(brush)

(earth+points).properties(width=1200, height=600) & (area_ufo_notselected + area_ufo_selected).properties(width=1200, height=100)